In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer

class QwenChatbot:
    def __init__(self, model_name="Qwen/Qwen3-0.6B"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.history = []

    def generate_response(self, user_input, think=False):
        messages = self.history + [{"role": "user", "content": user_input}]

        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=think,
        )

        inputs = self.tokenizer(text, return_tensors="pt")
        response_ids = self.model.generate(**inputs, max_new_tokens=32768)[0][len(inputs.input_ids[0]):].tolist()
        response = self.tokenizer.decode(response_ids, skip_special_tokens=True)

        # Update history
        self.history.append({"role": "user", "content": user_input})
        self.history.append({"role": "assistant", "content": response})

        return response

In [ ]:


chatbot = QwenChatbot()

instruction = """
你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称，月费价格，月流量。
根据用户输入，识别用户在上述三种属性上的倾向。
"""
input_text = """
办个100G的套餐
"""

prompt = f"""
{instruction}

用户输入：
{input_text}
"""

print(chatbot.generate_response(prompt))


根据用户输入“办个100G的套餐”，可以识别出以下三种属性的倾向：

1. **名称**：100G套餐  
2. **月费价格**：未明确提及，但100G套餐通常价格较高，可能为较高或中等价位  
3. **月流量**：100G流量  

因此，用户倾向于选择包含100G流量、较高月费价格的套餐。


In [6]:


chatbot = QwenChatbot()

output_format = """
以 JSON 格式输出，直接输出内容，不要用 markdown
"""

instruction = """
你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称，月费价格，月流量。
根据用户输入，识别用户在上述三种属性上的倾向。
"""
input_text = """
办个100G的套餐
"""

prompt = f"""
{instruction}

{output_format}

用户输入：
{input_text}
"""

print(chatbot.generate_response(prompt))


{
  "名称": "100G套餐",
  "月费价格": "未提供",
  "月流量": "100G"
}


In [19]:


chatbot = QwenChatbot()

output_format = """
以 JSON 格式输出，直接输出内容，不要用 markdown。

1. name 字段的取值为 string 类型，取值必须为以下之一：经济套餐、畅游套餐、无限套餐、校园套餐 或 null

2. price 字段的取值为一个结构体 或 null ，包含两个字段：
(1) operator, string 类型，取值范围： '<=' (小于等于) , '>=' (大于等于) , '==' (等于)
(2) value, int 类型

3. data 字段的取值为一个结构体 或 null ，包含两个字段：
(1) operator, string 类型，取值范围： '<=' (小于等于) , '>=' (大于等于) . '==' (等于)
(2) value, int 类型 或 string 类型, string 类型只能是'无上限'

4. 用户的效果图可以包含按 price 或 data 排序，以 sort 字段标识，取值为一个
(1) 结构体中以 "ordering"="descend" 表示按降序排序，以 "value" 字段存储待排序的字段
(2) 结构体中以 "ordering"="ascend" 表示按升序排序，以 "value" 字段存储待排序的字段

输出中只包含用户提及的字段，不要猜测任何用户未直接提及的字段，如果字段 value 为 null，则不输出该字段
"""

instruction = """
你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称(name)，月费价格(price)，月流量(data)。
根据用户输入，识别用户在上述三种属性上的倾向。
"""

# input_text = "办个100G以上的套餐"
# input_text = "我要无限量套餐"
input_text = "有没有更便宜的套餐"

prompt = f"""
{instruction}

{output_format}

用户输入：
{input_text}
"""

print(chatbot.generate_response(prompt, think=True))


<think>
好的，我现在需要分析用户输入的“有没有更便宜的套餐”来识别手机流量套餐产品的选择条件。首先，用户的问题是在询问是否有更便宜的套餐选择。根据题目要求，我需要检查用户提到的三个属性：名称、月费价格、月流量，以及可能的排序条件。

用户的问题本身并没有直接提到任何套餐的具体信息，比如名称、价格或流量数据。因此，我需要根据问题内容来推断可能的属性。通常，用户在询问价格时，会关注月费价格，而流量可能取决于用户是否需要额外的数据。但这里用户的问题只是询问是否有更便宜的套餐，没有具体说明价格或流量的具体数值。

接下来，我需要检查每个属性的取值范围是否符合用户的问题。例如，用户可能想比较不同套餐的月费价格，所以月费的价格字段应该取值为一个结构体，包含operator和value。同样，月流量的字段也需要符合结构体的要求，但用户的问题中没有提供具体的数值或操作符，因此可能没有具体的value字段。

另外，用户的问题没有提到任何排序条件，比如是否按价格或流量排序，但题目中允许用户输入排序条件。不过，用户的问题中并没有明确说明排序的方式，所以可能不需要处理排序部分。

最后，我需要确保输出的JSON只包含用户提及的字段，且value字段为null的情况不输出。因此，最终的JSON结构应该只包含name、price和data字段，但因为用户的问题中没有提供具体数值，所以price和data字段的value部分可能为null，但需要根据题目要求判断。根据题目描述，如果value为null则不输出该字段，所以price和data字段的value可能为null，但需要确认。不过用户的问题中并没有提供这些信息，所以可能无法确定。因此，最终的JSON输出将包含name字段为null，price和data字段的value为null。
</think>

{
  "name": null,
  "price": {
    "operator": "null",
    "value": null
  },
  "data": {
    "operator": "null",
    "value": null
  }
}


In [ ]:


chatbot = QwenChatbot()

examples = """
便宜的套餐： {"sort":{"ordering"="ascend","value"="price"}}
有没有不限流量的： {"data":{"operator":"==","value":"无上限"}}
流量大的： {"sort":{"ordering"="descend","value"="data"}}
月费不超过200的： {"price":{"operator":"<=","value":"200"}}
就要月费180那个套餐： {"price":{"operator":"<=","value":"200"}}
经济套餐： {"name":"经济套餐"}
"""

output_format = """
以 JSON 格式输出，直接输出内容，不要用 markdown。

1. name 字段的取值为 string 类型，取值必须为以下之一：经济套餐、畅游套餐、无限套餐、校园套餐 或 null

2. price 字段的取值为一个结构体 或 null ，包含两个字段：
(1) operator, string 类型，取值范围： '<=' (小于等于) , '>=' (大于等于) , '==' (等于)
(2) value, int 类型

3. data 字段的取值为一个结构体 或 null ，包含两个字段：
(1) operator, string 类型，取值范围： '<=' (小于等于) , '>=' (大于等于) . '==' (等于)
(2) value, int 类型 或 string 类型, string 类型只能是'无上限'

4. 用户的效果图可以包含按 price 或 data 排序，以 sort 字段标识，取值为一个
(1) 结构体中以 "ordering"="descend" 表示按降序排序，以 "value" 字段存储待排序的字段
(2) 结构体中以 "ordering"="ascend" 表示按升序排序，以 "value" 字段存储待排序的字段

输出中只包含用户提及的字段，不要猜测任何用户未直接提及的字段，如果字段 value 为 null，则不输出该字段
"""

instruction = """
你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称(name)，月费价格(price)，月流量(data)。
根据用户输入，识别用户在上述三种属性上的倾向。
"""

input_text = "办个100G以上的套餐"
# input_text = "我要无限量套餐"
# input_text = "有没有便宜的套餐"

prompt = f"""
{instruction}

{output_format}

例如：
{examples}

用户输入：
{input_text}
"""

print(chatbot.generate_response(prompt, think=True))
# print(chatbot.generate_response(prompt, think=False))
